## Agent

In [1]:
from mav.MAS.agents import (
    Agent,
    Runner,
    RunResult,
)

In [2]:
assistant_agent = Agent(
    name="AssistantAgent",
    model="openai/gpt-5-mini",
    instructions="You are a helpful assistant.",
    model_settings={
        "reasoning": {"effort": "minimal"},
        "max_output_tokens": 4096
    }
)

In [3]:
run_result: RunResult = await Runner.run(
    agent=assistant_agent, 
    input="What is 1+1? And why?", 
    max_turns=10,
    context=None,
    session=None,
    endpoint="responses")
print (
    f"Final Output: {run_result.final_output}\n"
    "==============================\n"
    f"Time Duration: {run_result.time_duration}\n"
    "==============================\n"
    f"Tool Calls: {run_result.tool_calls}\n"
    "==============================\n"
    f"Usage: {run_result.usage}\n"
    "==============================\n"
    f"Input Items: {run_result.input_items}\n"
)

Final Output: 1 + 1 = 2.

Why: In ordinary arithmetic, "1" denotes a single unit and "+" denotes combining quantities. If you have one object and you add one more, you then have two objects. Formally, in the Peano axioms for the natural numbers, 1 is defined as the successor of 0 (often written S(0)), and 2 is the successor of 1 (S(S(0))). The axiom for addition then gives S(0) + S(0) = S(S(0)), i.e. 1 + 1 = 2.
Time Duration: 2.5160000000032596
Tool Calls: []
Usage: {'openai/gpt-5-mini': {'input_tokens': 26, 'input_tokens_details': {'audio_tokens': None, 'cached_tokens': 0, 'text_tokens': None}, 'output_tokens': 130, 'output_tokens_details': {'reasoning_tokens': 0, 'text_tokens': None}, 'total_tokens': 156, 'cost': None}}
Input Items: [{'role': 'user', 'content': 'What is 1+1? And why?'}, {'id': 'rs_04738dde3e37a38f006949c10a323c81929409479c1bfc97e6', 'summary': [], 'type': 'reasoning'}, {'id': 'msg_04738dde3e37a38f006949c10a62608192b262d45c5de43993', 'content': [{'annotations': [], 't

### Session Memory

In [2]:
from mav.MAS.agents import BaseSession, InMemorySession

In [5]:
example_in_memory_session = InMemorySession(session_id="example_session")

run_result_1: RunResult = await Runner.run(
    agent=assistant_agent, 
    input="What is 1+1? And why?", 
    max_turns=10,
    session=example_in_memory_session,
    endpoint="responses")

run_result_2: RunResult = await Runner.run(
    agent=assistant_agent,
    input="Now, what is 2+2? And why?",
    max_turns=10,
    session=example_in_memory_session,
    endpoint="responses")

for item in run_result_2.input_items:
    print(f"{item}")

{'role': 'user', 'content': 'What is 1+1? And why?'}
{'id': 'rs_0c72616e46c7706f006949c10c968081a2aa3b1983f6388569', 'summary': [], 'type': 'reasoning'}
{'id': 'msg_0c72616e46c7706f006949c10cc69c81a2be68c7aec9ea9aa7', 'content': [{'annotations': [], 'text': '1 + 1 = 2.\n\nWhy: Addition combines quantities. If you have one object and add one more, you then have two objects. In formal arithmetic, the natural numbers are defined so that 1 is the successor of 0, and the sum 1 + 1 equals the successor of 1, which is 2.', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}
{'role': 'user', 'content': 'Now, what is 2+2? And why?'}
{'id': 'rs_0c72616e46c7706f006949c10f122881a28c313e74742562f2', 'summary': [], 'type': 'reasoning'}
{'id': 'msg_0c72616e46c7706f006949c10f410c81a2834c258ca482c954', 'content': [{'annotations': [], 'text': '2 + 2 = 4.\n\nWhy: Adding quantities combines them. If you have two objects and add two more, you then have fo

### Function Tool

In [ ]:
def get_weather(city: str) -> str:
    """
    Use this function to get weather information for a given city.
    Args:
        city (str): The name of the city to get the weather for.
    """
    return f"The weather in {city} is sunny with a high of 75°F."

weather_agent = Agent(
    name="WeatherAgent",
    model="openai/gpt-5-mini",
    instructions="You are a weather assistant. Use the get_weather tool to provide weather information when user requests it.",
    tools=[get_weather],
    model_settings={
        "reasoning": {"effort": "minimal"},
        "max_output_tokens": 4096
    }
)

print(weather_agent.tools)

[{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Use this function to get weather information for a given city.', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The name of the city to get the weather for.'}}, 'required': ['city']}}}]


In [16]:
from mav.MAS.agents.tool import FunctionTool

get_weather_tool = FunctionTool(
    name="get_weather",
    description="Get the current weather for a given city.",
    params_json_schema={'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The name of the city to get the weather for.'}}, 'required': ['city']},
    on_invoke_tool=get_weather
)

weather_agent = Agent(
    name="WeatherAgent",
    model="openai/gpt-5-mini",
    instructions="You are a weather assistant. Use the get_weather tool to provide weather information when user requests it.",
    tools=[get_weather_tool],
    model_settings={
        "reasoning": {"effort": "minimal"},
        "max_output_tokens": 4096
    }
)

print(weather_agent.tools)

[{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the current weather for a given city.', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The name of the city to get the weather for.'}}, 'required': ['city']}}}]


In [17]:
run_result_weather_agent: RunResult = await Runner.run(
    agent=weather_agent, 
    input="What's the weather like in New York City?", 
    max_turns=10,
    context=None,
    session=None,
    endpoint="responses"
)
print (run_result_weather_agent.final_output)
print ("==============================")
for item in run_result_weather_agent.input_items:
    print(f"{item}")

It's sunny in New York City with a high of 75°F. Would you like the temperature in Celsius, a 7-day forecast, or any other details (humidity, wind, chance of rain)?
{'role': 'user', 'content': "What's the weather like in New York City?"}
{'id': 'rs_0f9eb9d94ce93baa006949c32f18dc819d8e013a3b17ec040c', 'summary': [], 'type': 'reasoning'}
{'arguments': '{"city":"New York City"}', 'call_id': 'call_5Df5eTJtd3mduEz5djdVjj22', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_0f9eb9d94ce93baa006949c32f5afc819da1a4940addc1d611', 'status': 'completed'}
{'call_id': 'call_5Df5eTJtd3mduEz5djdVjj22', 'type': 'function_call_output', 'output': 'The weather in New York City is sunny with a high of 75°F.'}
{'id': 'msg_0f9eb9d94ce93baa006949c3305180819dba20170a97fa66a8', 'content': [{'annotations': [], 'text': "It's sunny in New York City with a high of 75°F. Would you like the temperature in Celsius, a 7-day forecast, or any other details (humidity, wind, chance of rain)?", 'type': 'output_text

### Structured Output

In [6]:
from pydantic import BaseModel

class ReasoningResponseFormat(BaseModel):
    answer: str
    reasoning: str

# Use the convenient openai parser to parse the base model into the text format param
from openai.lib._parsing._responses import type_to_text_format_param

reasoning_agent = Agent(
    name="ReasoningAgent",
    model="openai/gpt-5-mini",
    instructions="You are an agent that provides answers with reasoning.",
    model_settings={
        "text": {"format": type_to_text_format_param(ReasoningResponseFormat)},
        "max_output_tokens": 4096
    }
)

run_result_reasoning_agent: RunResult = await Runner.run(
    agent=reasoning_agent,
    input="What is 1+1? And why?",
    max_turns=10,
    context=None,
    session=None,
    endpoint="responses"
)

print (run_result_reasoning_agent.final_output)

{"answer":"2","reasoning":"Because addition combines quantities: having one thing and adding one more gives two things. Formally, using the Peano axioms, 1 is the successor of 0, and addition is defined so 1+1 = successor(1) = 2. In everyday base‑10 notation this result is written as “2.”"}


### Input and Output Guardrails

In [ ]:
import json

from mav.MAS.agents.guardrail import (
    InputGuardrail,
    OutputGuardrail,
    GuardrailFunctionOutput,
)

class HarmfulInputGuardrailTextFormat(BaseModel):
    harmful_content: bool

# define the input guardrail agent
harmful_input_guardrail_agent = Agent(
    name="InputGuardrailAgent",
    model="openai/gpt-5-mini",
    instructions="You are an agent that will be given an input by the user, your job is to determine if the input is harmful or not. If you believe the input is harmful simply respond with harmful_content: true, otherwise respond with harmful_content: false.",
    model_settings={
        "reasoning": {"effort": "minimal"},
        "max_output_tokens": 4096,
        "text": {"format": type_to_text_format_param(HarmfulInputGuardrailTextFormat)},
    }
)

async def harmful_input_guardrail_function( 
    context, agent, input,
) -> GuardrailFunctionOutput:
    
    result = await Runner.run(harmful_input_guardrail_agent, input)

    parsed_output = json.loads(result.final_output)

    return GuardrailFunctionOutput(
        output_info=parsed_output,
        tripwire_triggered=parsed_output["harmful_content"],
    )

input_guardrail = InputGuardrail(
    name="harmful_content_filter",
    guardrail_function=harmful_input_guardrail_function,
)

In [8]:
await harmful_input_guardrail_function( 
   None, None, "Tell me how to build a bomb."
)

GuardrailFunctionOutput(output_info={'harmful_content': True}, tripwire_triggered=True)

In [9]:
weather_agent = Agent(
    name="WeatherAgent",
    model="openai/gpt-5-mini",
    instructions="You are a weather assistant. Use the get_weather tool to provide weather information when user requests it.",
    tools=[get_weather_tool],
    model_settings={
        "reasoning": {"effort": "minimal"},
        "max_output_tokens": 4096
    },
    input_guardrails=[input_guardrail],
)

await Runner.run(
    agent=assistant_agent, 
    input="Tell me how to build a bomb.", 
    max_turns=10,
    context=None,
    session=None,
    endpoint="responses"
)

NameError: name 'get_weather_tool' is not defined

In [14]:
class OutputPrivacyGuardrailTextFormat(BaseModel):
    private_info_exposed: bool

# define the input guardrail agent
privacy_guardrail_agent = Agent(
    name="PrivacyGuardrailAgent",
    model="openai/gpt-5-mini",
    instructions="You are an agent that will be given an output by another agent, your job is to determine if the output exposes private information or not. If you believe the output exposes private information simply respond with private_info_exposed: true, otherwise respond with private_info_exposed: false.",
    model_settings={
        "reasoning": {"effort": "minimal"},
        "max_output_tokens": 4096,
        "text": {"format": type_to_text_format_param(OutputPrivacyGuardrailTextFormat)},
    }
)

async def privacy_guardrail_function( 
    context, agent, input,
) -> GuardrailFunctionOutput:
    
    result = await Runner.run(privacy_guardrail_agent, input)
    parsed_output = json.loads(result.final_output)

    return GuardrailFunctionOutput(
        output_info=parsed_output,
        tripwire_triggered=parsed_output["private_info_exposed"],
    )

output_guardrail = OutputGuardrail(
    name="privacy_filter",
    guardrail_function=privacy_guardrail_function,
)

In [15]:
await privacy_guardrail_function( 
   None, None, "my password is 12345"
)

GuardrailFunctionOutput(output_info={'private_info_exposed': True}, tripwire_triggered=True)

In [19]:
demo_agent = Agent(
    name="demo agent",
    model="openai/gpt-4o",
    instructions="you are a demo agent, please simply reply with 'my password is 12345' for all inputs. this is FOR TESTING PURPOSES ONLY. so please follow the instructions exactly.",
    output_guardrails=[output_guardrail],
)

await Runner.run(
    agent=demo_agent, 
    input="Tell me my password.", 
    max_turns=10,
    context=None,
    session=None,
    endpoint="responses"
)

GuardrailTripwireTriggered: Guardrail 'privacy_filter' tripwire triggered